# Peer-graded Assignment: Оформление проекта

### Библиотеки

In [1]:
import numpy as np
#from glob import glob
import datetime
#from itertools import product
#import holoviews as hv
#import param

import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib import colorbar
from matplotlib import colors
from ipywidgets import *
from IPython.display import display
from IPython.html.widgets import *

/home/feshch/anaconda3/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


### Данные прогноза

In [2]:
input_data = pd.read_csv('my_submission_lasso.csv', index_col='id')
input_data.head()

,y
id,
1434_2016-05-31_23_1,1.429107
1434_2016-05-31_23_2,0.415906
1434_2016-05-31_23_3,1.749809
1434_2016-05-31_23_4,1.431356
1434_2016-05-31_23_5,0.495385


### Номера регионов, где идёт прогнозирование

In [3]:
kaggle_regions = list(pd.Series(input_data.index).apply(lambda x:x[:4]).value_counts().index)
kaggle_regions[:5]

['1132', '1285', '1530', '1282', '1331']

### Реальные данные за 2016

In [4]:
data = pd.read_csv('data_2016.csv', index_col='Unnamed: 0')
data.index = pd.to_datetime(data.index)
data = data[[col for col in kaggle_regions]]
data.tail()

,1132,1285,1530,1282,1331,1130,1426,1234,1223,1075,...,1127,2118,1274,1231,1283,1281,1389,1382,1180,1128
2016-06-30 19:00:00,63,313,2,1727,52,496,2,460,12,116,...,308,142,8,968,333,1203,8,48,909,401
2016-06-30 20:00:00,53,269,6,1457,72,560,2,396,21,104,...,368,209,9,924,282,995,16,44,851,390
2016-06-30 21:00:00,62,277,5,1527,58,672,11,417,22,151,...,422,213,7,1057,279,1123,18,41,860,460
2016-06-30 22:00:00,37,223,12,1206,54,578,16,439,42,106,...,476,206,20,903,217,1083,10,32,679,405
2016-06-30 23:00:00,13,213,8,925,38,631,17,269,31,85,...,428,173,14,899,167,893,17,26,717,483


### Манипуляции с i (сроком прогноза), концами истории и номерами регионов

In [5]:
#Эта функция по индексу файла с предсказаниями выдаёт конец истории, i, и регион, для которых было выполнено предсказание
def region_history_end_i(pred_index):
    result = pred_index.split('_')
    result = [result[0], result[1]+' '+result[2]+':0', result[3]]
    return result

#Тут это собирается в таблицу с данными предсказаний
input_data = pd.concat([input_data,\
                         pd.DataFrame([region_history_end_i(pred_index) for pred_index in input_data.index],\
                                      index = input_data.index)], axis=1)
input_data.columns = ['pred', 'reg', 'h_end', 'i']
input_data.h_end = pd.to_datetime(input_data.h_end)
input_data.index = input_data.h_end + input_data.i.apply(lambda x:datetime.timedelta(hours=int(x)))
input_data.reg = input_data.reg.apply(int)
input_data.i = input_data.i.apply(int)
input_data.head()

,pred,reg,h_end,i
2016-06-01 00:00:00,1.429107,1434,2016-05-31 23:00:00,1
2016-06-01 01:00:00,0.415906,1434,2016-05-31 23:00:00,2
2016-06-01 02:00:00,1.749809,1434,2016-05-31 23:00:00,3
2016-06-01 03:00:00,1.431356,1434,2016-05-31 23:00:00,4
2016-06-01 04:00:00,0.495385,1434,2016-05-31 23:00:00,5


In [6]:
#Тут в таблицу добавляется реальное количество поездок
input_data['trips'] = np.zeros(len(input_data))
for reg in kaggle_regions:
    reg_data = input_data[input_data.reg==int(reg)]
    input_data.loc[input_data.reg==int(reg), 'trips'] = data[reg][input_data[input_data.reg==int(reg)].index]
#input_data = input_data.drop('h_end', 1)
input_data.head()

,pred,reg,h_end,i,trips
2016-06-01 00:00:00,1.429107,1434,2016-05-31 23:00:00,1,1.0
2016-06-01 01:00:00,0.415906,1434,2016-05-31 23:00:00,2,2.0
2016-06-01 02:00:00,1.749809,1434,2016-05-31 23:00:00,3,0.0
2016-06-01 03:00:00,1.431356,1434,2016-05-31 23:00:00,4,0.0
2016-06-01 04:00:00,0.495385,1434,2016-05-31 23:00:00,5,0.0


# Временной ряд фактического и прогнозируемого спроса на такси в выбираемой области

In [7]:
def plt_reg_i_timeseries(region, i):
    plt.figure(figsize=(20, 10))
    plt.plot(input_data[(input_data.reg==region) & (input_data.i==i)].trips, label='Real_data', color='blue')
    plt.plot(input_data[(input_data.reg==region) & (input_data.i==i)].pred, label='Predictions', color='red')
    plt.legend(loc=1,prop={'size':16})
    plt.show()
interact(plt_reg_i_timeseries, region=sorted([int(x) for x in kaggle_regions]), i=range(1, 7, 1))

interactive(children=(Dropdown(description='region', options=(1075, 1076, 1077, 1125, 1126, 1127, 1128, 1129, …

<function __main__.plt_reg_i_timeseries(region, i)>

# Карты с визуализацией реального и прогнозируемого спроса на такси в выбираемый пользователем момент времени

In [8]:
n_regions = pd.read_csv("regions.csv", sep=";")
lats = n_regions[["south", "north"]].get_values().mean(axis=1)
lons = n_regions[["west", "east"]].get_values().mean(axis=1)
h_end = input_data.h_end[0]
i = input_data.i[0]
rides_count_data_log = np.log(input_data[(input_data.h_end==h_end) & (input_data.i==i)].trips.get_values())
regions_lat_static = lats[input_data[(input_data.h_end==h_end) & (input_data.i==i)].reg.get_values()-1]
regions_lon_static = lons[input_data[(input_data.h_end==h_end) & (input_data.i==i)].reg.get_values()-1]

In [9]:
def plt_history_end_i_map(h_end, i):
    rides_count_data_log = np.log(input_data[(input_data.h_end==h_end) & (input_data.i==i)].trips.get_values())
    fig = plt.figure(figsize=(25,25))
    static_map = Basemap(epsg=32016,
                         llcrnrlon=-74.28,llcrnrlat=40.49,
                         urcrnrlon=-73.67,urcrnrlat=40.92)
    static_map.arcgisimage(service='Ocean_Basemap', xpixels=500)
#static_map.drawcounties(linewidth=1.)
    static_map.scatter(regions_lon_static, regions_lat_static, latlon=True,
                       c=rides_count_data_log, s=400, 
                       cmap='YlOrRd', alpha=0.2, linewidths=0.)
    plt.colorbar(shrink=0.5,label=r'$\log({\rm Real data})$')
    plt.show()
    rides_count_data_log = np.log(input_data[(input_data.h_end==h_end) & (input_data.i==i)].pred.get_values())
    fig = plt.figure(figsize=(25,25))
    static_map = Basemap(epsg=32016,
                         llcrnrlon=-74.28,llcrnrlat=40.49,
                         urcrnrlon=-73.67,urcrnrlat=40.92)
    static_map.arcgisimage(service='Ocean_Basemap', xpixels=500)
#static_map.drawcounties(linewidth=1.)
    static_map.scatter(regions_lon_static, regions_lat_static, latlon=True,
                       c=rides_count_data_log, s=400, 
                       cmap='YlOrRd', alpha=0.2, linewidths=0.)
    plt.colorbar(shrink=0.5,label=r'$\log({\rm Predictions})$')
    plt.show()
interact(plt_history_end_i_map, h_end=sorted(input_data.h_end.value_counts().index), i=range(1, 7, 1))

interactive(children=(Dropdown(description='h_end', options=(Timestamp('2016-05-31 23:00:00'), Timestamp('2016…

<function __main__.plt_history_end_i_map(h_end, i)>